 Prepare the Iris Dataset

In [ ]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import mlflow
import mlflow.sklearn

iris = load_iris()
X = pd.DataFrame(iris.data, columns=iris.feature_names)
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


Train and save Models with MLflow

In [ ]:
mlflow.sklearn.autolog()

mlflow.set_tracking_uri('http://localhost:5000') 
with mlflow.start_run():
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy}")

    mlflow.log_metric("accuracy", accuracy)

    best_accuracy = 0.9  
    if accuracy > best_accuracy:
        mlflow.sklearn.log_model(model, "model", registered_model_name="IrisClassifier")

        print("New best model registered!")
    else:
        print("Current model did not outperform the best model.")

Accuracy: 1.0


2024/12/01 16:52:14 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'IrisClassifier'.
2024/12/01 16:52:14 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: IrisClassifier, version 1


New best model registered!
🏃 View run nervous-tern-180 at: http://localhost:5000/#/experiments/0/runs/453a6cdb21fa4c50b8386be0676422f4
🧪 View experiment at: http://localhost:5000/#/experiments/0


Created version '1' of model 'IrisClassifier'.


In [ ]:
import random
mlflow.sklearn.autolog()

mlflow.set_tracking_uri('http://localhost:5000')

best_accuracy = 0.9  

for experiment_num in range(1, 6): 
    with mlflow.start_run():
        accuracy = random.uniform(0.5, 0.7) + (experiment_num * 0.05)  
        
        model = RandomForestClassifier(n_estimators=100, random_state=42)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        print(f"Experiment {experiment_num} - Accuracy: {accuracy}")

        mlflow.log_metric("accuracy", accuracy)

        if accuracy > best_accuracy:
            mlflow.sklearn.log_model(model, "model", registered_model_name="IrisClassifier")
            print(f"Experiment {experiment_num}: New best model registered with accuracy {accuracy}!")
            best_accuracy = accuracy  
        else:
            print(f"Experiment {experiment_num}: Current model did not outperform the best model.")

In [ ]:
model_name = "IrisClassifier"
model_stage = "Production"
print("MLflow Tracking URI:", mlflow.get_tracking_uri())

client = MlflowClient()

try:
    versions = client.get_latest_versions(model_name, stages=[model_stage])
    if versions:
        print(f"Model {model_name} is already in stage {model_stage}")
    else:
        print(f"No versions of model with name '{model_name}' and stage '{model_stage}' found.")
        latest_version = client.get_latest_versions(model_name)[0].version
        
        client.transition_model_version_stage(
            name=model_name,
            version=latest_version,
            stage=model_stage
        )
        print(f"Model version {latest_version} has been moved to '{model_stage}' stage.")

except mlflow.exceptions.MlflowException as e:
    print(f"Error: {e}")


MLflow Tracking URI: http://localhost:5000


C:\Users\tahaf\AppData\Local\Temp\ipykernel_33244\2075413859.py:11: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  versions = client.get_latest_versions(model_name, stages=[model_stage])


Model IrisClassifier is already in stage Production
